### Using IPUMS records, calculate in-migration and out-migration to/from NYC - NYC Metro and domestic U.S.

Requires download of csv extracts from IPUMS USA website
https://usa.ipums.org

For more information about specific variables available for download, refer to IPUMS-USA website, ex.:
https://usa.ipums.org/usa-action/variables/MIGRATE1#codes_section

This notebook updates the previous to pull 4 1-year Estimates for 2016 to 2019

In [1]:
import pandas as pd
import numpy as np
import math

#### Stat functions for using replicate weights

In [2]:
# functions to calculate standard error, moe, and coefficient of variation
def get_se(per_wt,rep_weights):
    result = math.sqrt((sum(map(lambda x: (x-per_wt)**2,rep_weights))/20))
    return result

def get_moe(se):
    return se*1.645 #90% confidence interval

def agg_moe(m):
    result = math.sqrt(sum(map(lambda x: x**2, m)))
    return result

def get_cv(est,m):
    if est == 0:
        return 0
    else:
        return (np.absolute(m/1.645/est))*100

In [3]:
# create a list of replicate weights
repwt = 'REPWTP'
repwts = [repwt+str(i) for i in range(1, 81)]

#### Geography look up files & cleanup

In [4]:
# pull in geography reference files
df_migpuma = pd.read_csv('../data/migpwpuma_xwalk_10.csv')
df_respuma = pd.read_csv('../data/respuma_xwalk_10.csv')
df_migpl = pd.read_csv('../data/migpl_xwalk.csv')

#df_migpuma.head()

In [5]:
# clean up migpuma table for merge with data table 
df_migpuma['MIGPUMA_str'] = df_migpuma['MIGPUMAID'].apply(str).apply(lambda x: '{0:0>8}'.format(x))
df_migpuma['CountyFIP'] = df_migpuma['CountyFIP'].apply(str)
df_migpuma = df_migpuma.drop(columns=['STATEFIP','State','MIGSTATE','MIGPUMA','PWSTATE','PWPUMA','StringGIS'])
#df_migpuma.head()

In [6]:
df_respuma['puma_id'] = df_respuma['GEOID10'].apply(str).apply(lambda x: '{0:0>8}'.format(x))
df_respuma['CountyFIPS'] = df_respuma['CountyFIPS'].apply(str).apply(lambda x: '{0:0>5}'.format(x))
df_respuma = df_respuma.drop(columns=['StateFIPS','PUMA'])

In [7]:
#forgot to pull COUNTYFIPS in pums extract for in-migration
#these are to clean the in-migration table for borough totals
nyc_county = dict(df_respuma[['puma_id','CountyFIPS']].values)
nyc_respuma = df_respuma[df_respuma['Subregion']=='NYC']['puma_id'].tolist()

### In-migrants to NYC boroughs by NYC Metro Subregion & County
Following options available:
- By NYC total or Borough of Current Residence (i.e. destination)
- BY NYC Metro Subregion of residence 1 year ago (i.e. origin)

In [8]:
# read in full inflow table (raw PUMS data)
df = pd.read_csv(f'../data/nys_in_21.csv.gz',compression='gzip')
#df.head()

In [9]:
# create a new id columns for merge with geo lookup tables
df['migpuma_id'] = df['MIGPLAC1'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    df['MIGPUMA1'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

df['in_respuma'] = df['STATEFIP'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                   df['PUMA'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

In [10]:
# REDUCE DATA TABLE TO PEOPLE WHO HAVE MOVED TO NYC COUNTIES ONLY IN THE LAST YEAR
# Select only records for people who have moved in the last year
# MIGRATE1 == 2,3,4 ; other codes are people who haven't moved
mig_codes = [2,3,4]
df = df[df['MIGRATE1'].isin(mig_codes)]

# Select records for people who live in NYC currently
df = df[df['in_respuma'].isin(nyc_respuma)]
df['in_stco'] = df.in_respuma.map(nyc_county)
df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,HHTYPE,REPWT,CLUSTER,STATEFIP,PUMA,...,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80,migpuma_id,in_respuma,in_stco
5,2021,202101,843587,2021010000099,109.0,0,1,2021008435871,36,3710,...,111,112,107,108,109,25,25,03603700,03603710,36005
48,2021,202101,843630,2021010000623,21.0,0,1,2021008436301,36,4106,...,3,39,21,4,3,21,20,43800001,03604106,36081
52,2021,202101,843634,2021010000662,68.0,0,1,2021008436341,36,3704,...,68,9,11,69,67,67,10,03603700,03603704,36005
57,2021,202101,843639,2021010000701,69.0,0,1,2021008436391,36,3703,...,69,69,130,69,70,72,70,03604100,03603703,36005
63,2021,202101,843645,2021010000752,34.0,0,1,2021008436451,36,3802,...,61,34,34,62,5,34,34,03401300,03603802,36061


In [11]:
# merge with geography look up tables 
df_in = df.merge(df_migpl,how='left',left_on='MIGPLAC1',right_on='migplac_id').merge(df_migpuma,how='left',left_on='migpuma_id',right_on='MIGPUMA_str')
#df_in.head()

In [12]:
# replace NaN values in aggregation columns with domestic/intl values
column_clean = ['Subregion5','Subregion7','Region','County','CountyFIP']
for i in column_clean:
    df_in.loc[df_in[i].isnull(),i] = df_in['US_intl']

In [13]:
# reduce table to just counties and subregions of residence 1 year ago
cols = ['YEAR','in_stco','CountyFIP','Subregion7','Subregion5','Region','PERWT'] + repwts
df_in = df_in[cols]

# rename for clarity
df_in = df_in.rename(columns={'County':'out_co_name','CountyFIP':'out_stco',\
                              'Subregion5':'out_subreg_5','Subregion7':'out_subreg_7',\
                              'Region':'out_reg','PERWT':'in_pop'})
    
df_in.head()

,YEAR,in_stco,out_stco,out_subreg_7,out_subreg_5,out_reg,in_pop,REPWTP1,REPWTP2,REPWTP3,...,REPWTP71,REPWTP72,REPWTP73,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80
0,2021,36005,36005,NYC,NYC,NYC,109.0,109,202,26,...,104,192,193,111,112,107,108,109,25,25
1,2021,36081,intl,intl,intl,intl,21.0,3,3,41,...,21,3,21,3,39,21,4,3,21,20
2,2021,36005,36005,NYC,NYC,NYC,68.0,10,69,10,...,67,11,11,68,9,11,69,67,67,10
3,2021,36005,36081,NYC,NYC,NYC,69.0,10,127,129,...,69,73,131,69,69,130,69,70,72,70
4,2021,36061,34013,NJ In,NJ,Region,34.0,5,7,36,...,5,34,63,61,34,34,62,5,34,34


#### NYC Boroughs by Subregion of Origin (simplified) table

Select different summary columns in groupby function to choose counties or aggregation by borough of current residence.

In [14]:
#CHANGE TO PIVOT TABLE AND THEN RESET INDEX

dff_in = df_in.groupby(['YEAR','in_stco','out_reg']).sum().reset_index()

In [15]:
# calculate standard error, margin of error, cv
# drop replicate weight columns
dff_in['in_se'] = dff_in.apply(lambda x: (get_se(x['in_pop'],x[repwts])),axis=1)
dff_in['in_moe'] = dff_in.apply(lambda x: (get_moe(x['in_se'])),axis=1)
dff_in['in_cv'] = dff_in.apply(lambda x: (get_cv(x['in_pop'],x['in_se'])),axis=1)

dff_in = dff_in.drop(columns=repwts) 
                            
dff_in

,YEAR,in_stco,out_reg,in_pop,in_se,in_moe,in_cv
0,2021,36005,NYC,77437.0,5772.019724,9494.972447,4.531202
1,2021,36005,Region,5488.0,1161.268660,1910.286945,12.863309
2,2021,36005,US,6782.0,1092.007189,1796.351825,9.788177
3,2021,36005,intl,7413.0,1335.816548,2197.418222,10.954358
4,2021,36047,NYC,226099.0,8916.872465,14668.255204,2.397441
5,2021,36047,Region,12987.0,1777.142425,2923.399290,8.318547
6,2021,36047,US,30335.0,2498.849905,4110.608094,5.007607
7,2021,36047,intl,8973.0,1382.012138,2273.409967,9.362855
8,2021,36061,NYC,166699.0,8532.327051,14035.677998,3.111491
9,2021,36061,Region,35209.0,3123.250198,5137.746576,5.392463


### NYC resident out-migration to NYC Metro and U.S.

##### update csv paths & keep aggregation geography consistent with previous for net flow table in following section

In [16]:
# pull in outflow table
dff = pd.read_csv(f'../data/nys_out_21.csv.gz',compression='gzip')

In [17]:
nyc_mig = {'03603700':'36005','03603800':'36061','03603900':'36085','03604000':'36047','03604100':'36081'}

In [18]:
# create new id columns for merge with geo lookup table
dff['in_respuma'] = dff['STATEFIP'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    dff['PUMA'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

dff['out_migpuma'] = dff['MIGPLAC1'].apply(str).apply(lambda x: '{0:0>3}'.format(x)) + \
                    dff['MIGPUMA1'].apply(str).apply(lambda x: '{0:0>5}'.format(x))

In [19]:
# Select records only for people who lived in NYC one year ago (NYC out-migrants)
dff = dff[dff['out_migpuma'].isin(nyc_mig.keys())]
dff['out_stco'] = dff.out_migpuma.map(nyc_mig)

In [20]:
# merge with geography look up tables 
df_out = dff.merge(df_respuma,how='left',left_on='in_respuma',right_on='puma_id')

In [21]:
# replace NaN values in aggregation columns with US for non region
column_clean = ['NAME','Subregion','Subregion2','Region','County','CountyFIPS']
for i in column_clean:
    df_out.loc[df_out[i].isnull(),i] = 'US'

In [22]:
df_out.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,HHTYPE,REPWT,CLUSTER,STATEFIP,PUMA,...,out_migpuma,out_stco,GEOID10,NAME,Subregion,Subregion2,Region,County,CountyFIPS,puma_id
0,2021,202101,160,2021010010833,59.0,0,1,2021000001601,1,2500,...,03603900,36085,NaN,US,US,US,US,US,US,NaN
1,2021,202101,3717,2021000071776,57.0,1,1,2021000037171,1,600,...,03604000,36047,NaN,US,US,US,US,US,US,NaN
2,2021,202101,3717,2021000071776,57.0,1,1,2021000037171,1,600,...,03604000,36047,NaN,US,US,US,US,US,US,NaN
3,2021,202101,10050,2021000506283,51.0,6,1,2021000100501,1,1302,...,03603800,36061,NaN,US,US,US,US,US,US,NaN
4,2021,202101,10682,2021000550134,62.0,6,1,2021000106821,1,2000,...,03604000,36047,NaN,US,US,US,US,US,US,NaN


In [23]:
# reduce table to just current counties and subregions of residence 
cols = ['YEAR','County','CountyFIPS','Subregion','Subregion2','Region','out_stco','PERWT'] + repwts
df_out = df_out[cols]

# rename for clarity
df_out = df_out.rename(columns={'County':'in_co_name','CountyFIPS':'in_stco',\
                              'Subregion':'in_subreg_7','Subregion2':'in_subreg_5',\
                              'Region':'in_reg','PERWT':'out_pop'})
    
df_out.head()

,YEAR,in_co_name,in_stco,in_subreg_7,in_subreg_5,in_reg,out_stco,out_pop,REPWTP1,REPWTP2,...,REPWTP71,REPWTP72,REPWTP73,REPWTP74,REPWTP75,REPWTP76,REPWTP77,REPWTP78,REPWTP79,REPWTP80
0,2021,US,US,US,US,US,36085,59.0,13,63,...,11,52,87,48,45,110,57,7,7,12
1,2021,US,US,US,US,US,36047,56.0,76,62,...,76,65,51,83,54,15,58,94,116,61
2,2021,US,US,US,US,US,36047,38.0,50,48,...,47,31,40,83,39,11,48,82,68,50
3,2021,US,US,US,US,US,36061,51.0,14,97,...,15,57,53,42,48,14,57,51,89,51
4,2021,US,US,US,US,US,36047,62.0,63,15,...,75,17,19,102,68,121,77,63,98,68


#### NYC total outflow by Subregion Destination (simplified) table

Select different summary columns in groupby function to choose counties or aggregation by borough of previous residence.

In [24]:
dff_out = df_out.groupby(['YEAR','out_stco','in_reg']).sum().reset_index()
#dff_out

In [25]:
# calculate standard error, margin of error, cv
# drop replicate weight columns
dff_out['out_se'] = dff_out.apply(lambda x: (get_se(x['out_pop'],x[repwts])),axis=1)
dff_out['out_moe'] = dff_out.apply(lambda x: (get_moe(x['out_se'])),axis=1)
dff_out['out_cv'] = dff_out.apply(lambda x: (get_cv(x['out_pop'],x['out_se'])),axis=1)

dff_out = dff_out.drop(columns=repwts) 
                            
dff_out.head()

,YEAR,out_stco,in_reg,out_pop,out_se,out_moe,out_cv
0,2021,36005,NYC,72087.0,5540.177624,9113.592191,4.671978
1,2021,36005,Region,26709.0,3485.987837,5734.449992,7.934185
2,2021,36005,US,28761.0,3759.975146,6185.159115,7.947217
3,2021,36047,NYC,224966.0,9519.322828,15659.286052,2.572310
4,2021,36047,Region,50051.0,5241.877851,8622.889064,6.366610


## Export in and out table to Excel

In [26]:
with pd.ExcelWriter(f'../output/nycmig_inout_reg_boro_2021.xlsx') as writer:
    dff_in.to_excel(writer,sheet_name="In_mig", index=False)
    dff_out.to_excel(writer,sheet_name="Out_mig", index=False)

## Pivot & resave

In [27]:
dfff_in = pd.pivot_table(dff_in,values=['in_pop','in_moe','in_cv'],index='in_stco',\
                         columns=['YEAR','out_reg'],aggfunc=np.sum)

In [28]:
dfff_in

in_cv                                         in_moe               \
YEAR         2021                                           2021                
out_reg       NYC     Region         US       intl           NYC       Region   
in_stco                                                                         
36005    4.531202  12.863309   9.788177  10.954358   9494.972447  1910.286945   
36047    2.397441   8.318547   5.007607   9.362855  14668.255204  2923.399290   
36061    3.111491   5.392463   4.121768   8.389766  14035.677998  5137.746576   
36081    3.378296   9.128752   6.816534   9.080607  12702.467171  3126.858972   
36085    6.898393  17.893175  18.146327  22.016919   4841.717912   514.213818   

                                     in_pop                             
YEAR                                   2021                             
out_reg           US         intl       NYC   Region       US     intl  
in_stco                                                                 
36005    1796.351825  2197.418222   77437.0   5488.0   6782.0   7413.0  
36047    4110.608094  2273.409967  226099.0  12987.0  30335.0   8973.0  
36061    6971.673518  5838.055265  166699.0  35209.0  62506.0  25715.0  
36081    2896.714499  2597.788823  138950.0  12658.0  15704.0  10572.0  
36085    1082.261302   397.983265   25937.0   1062.0   2204.0    668.0